<a href="https://colab.research.google.com/github/SaakshiNarula/CommonSense_Question_Answering/blob/main/model_trials_NLP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  print("Using the GPU!")
  torch.backends.cudnn.enabled = True
  torch.backends.cudnn.benchmark = True
else:
  print("WARNING: Could not find GPU! Using CPU only. If you want to enable GPU, please to go Edit > Notebook Settings > Hardware Accelerator and select GPU.")


## Dataset 

In [ ]:
#upload training dataset

import json

dataset = []
for line in open('/content/drive/My Drive/EECS 595/Project/train_rand_split.json', 'r'):
    dataset.append(json.loads(line))


In [ ]:
#upload validation dataset

dataset_val = []
for line in open('/content/drive/My Drive/EECS 595/Project/dev_rand_split.json', 'r'):
    dataset_val.append(json.loads(line))


In [ ]:
dataset[10]

{'answerKey': 'B',
 'id': '1a7f5b3c65364d9be002576660c914fe',
 'question': {'choices': [{'label': 'A', 'text': 'mouth'},
   {'label': 'B', 'text': 'grocery cart'},
   {'label': 'C', 'text': 'super market'},
   {'label': 'D', 'text': 'fruit basket'},
   {'label': 'E', 'text': 'fruit market'}],
  'question_concept': 'grape',
  'stem': 'Where do you put your grapes just before checking out?'}}

In [ ]:
#creating train dataframe

import json
import pandas as pd

max_length = 0
option2label = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}

dataset_df_val = pd.DataFrame(columns=['question', 'A', 'B', 'C', 'D', 'E'])
for j_obj in dataset_val:
  qa = j_obj['question']
  label = j_obj['answerKey']
  options = qa['choices']

  row = pd.DataFrame(
    {"question": [qa['stem']],
      "A": [options[0]['text']],
      "B": [options[1]['text']],
      "C": [options[2]['text']],
      "D": [options[3]['text']],
      "E": [options[4]['text']],
      "label": int(option2label[label]),
      "idx": [j_obj['id']]})
  dataset_df_val = pd.concat([dataset_df_val, row])
  if len(qa['stem'].strip().split()) > max_length:
    max_length = len(qa['stem'].strip().split())

max_length

51

In [ ]:
#creating validation dataframe

import json
import pandas as pd

max_length = 0
option2label = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}

dataset_df = pd.DataFrame(columns=['question', 'A', 'B', 'C', 'D', 'E'])
for j_obj in dataset:
  qa = j_obj['question']
  label = j_obj['answerKey']
  options = qa['choices']

  row = pd.DataFrame(
    {"question": [qa['stem']],
      "A": [options[0]['text']],
      "B": [options[1]['text']],
      "C": [options[2]['text']],
      "D": [options[3]['text']],
      "E": [options[4]['text']],
      "label": int(option2label[label]),
      "idx": [j_obj['id']]})
  dataset_df = pd.concat([dataset_df, row])
  if len(qa['stem'].strip().split()) > max_length:
    max_length = len(qa['stem'].strip().split())

max_length

63

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
df_train = dataset_df
df_val = dataset_df_val

In [ ]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [ ]:
from transformers import RobertaTokenizer, RobertaForMultipleChoice
from transformers import BertTokenizer, BertForMultipleChoice
from transformers import AlbertTokenizer, AlbertForMultipleChoice
class CustomDataset(Dataset):

    def __init__(self, data, maxlen, bert_model='albert-base-v2'):

        self.data = data  # pandas dataframe
        #Initialize the tokenizer
        #self.tokenizer = AutoTokenizer.from_pretrained(bert_model)  
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        #self.tokenizer = BertTokenizer.from_pretrained(bert_model)

        self.maxlen = maxlen

    def __len__(self):
        return len(self.data)
    
    def get_embedding(self, index, question, option):
        ans = str(self.data.loc[index, option])
        #.encode_plus
        encoded_pair = self.tokenizer(
            'Q '+question, 'A '+ans,
            add_special_tokens=True,
            max_length=self.maxlen,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        
        token_ids = encoded_pair['input_ids']  # tensor of token ids
        attn_masks = encoded_pair['attention_mask'] # binary tensor with "0" for padded values and "1" for the other values
        token_type_ids = encoded_pair['token_type_ids']  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

        return torch.tensor(token_ids, dtype=torch.long), torch.tensor(attn_masks, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long)
      

    def __getitem__(self, index):
        # initialize the tensor vectors
        question = str(self.data.loc[index, 'question'])
        tid, am, ttid = self.get_embedding(index, question, 'A')
        token_ids_tensor = tid.unsqueeze(0)
        attn_masks_tensor = am.unsqueeze(0)
        token_type_ids_tensor = ttid.unsqueeze(0)

        # Selecting Q and A at the specified index in the data frame
        for option in ['B', 'C', 'D', 'E']:
          tid, am, ttid = self.get_embedding(index, question, option)
          token_ids_tensor = torch.cat([token_ids_tensor, tid.unsqueeze(0)], dim=0)
          attn_masks_tensor = torch.cat([attn_masks_tensor, am.unsqueeze(0)], dim=0)
          token_type_ids_tensor = torch.cat([token_type_ids_tensor, ttid.unsqueeze(0)], dim=0)

        # print(token_ids_tensor[0])
        return {
            'input_ids': token_ids_tensor,
            'attention_mask': attn_masks_tensor,
            'token_type_ids': token_type_ids_tensor,
            'labels': torch.tensor(self.data.loc[index, 'label'], dtype=torch.long)
        }

## Model

In [ ]:
class SentencePairClassifier(nn.Module):

    def __init__(self, bert_model="albert-base-v2", freeze_bert=False):
        super(SentencePairClassifier, self).__init__()
        #  Instantiating BERT-based model object
        self.bert_layer = RobertaForMultipleChoice.from_pretrained(bert_model)
        #AutoModel.from_pretrained(bert_model)

        #  Fix the hidden-state size of the encoder outputs (If you want to add other pre-trained models here, search for the encoder output size)
        if bert_model == "albert-base-v2" or bert_model == "roberta-base":  # 12M parameters
            hidden_size = 768
        elif bert_model == "albert-large-v2":  # 18M parameters
            hidden_size = 1024
        elif bert_model == "albert-xlarge-v2":  # 60M parameters
            hidden_size = 2048
        elif bert_model == "albert-xxlarge-v2":  # 235M parameters
            hidden_size = 4096
        elif bert_model == "bert-base-uncased": # 110M parameters
            hidden_size = 768
        elif bert_model == "roberta-large-mnli" or bert_model == "roberta-large":
          hidden_size = 1024

        # Freeze bert layers and only train the classification layer weights
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = True

        # Classification layer
        self.cls_layer = nn.Linear(hidden_size, 1)

        self.dropout = nn.Dropout(p=0.5)

    @autocast()  # run in mixed precision
    def forward(self, input_ids, attn_masks, token_type_ids):
        '''
        Inputs:
            -input_ids : Tensor  containing token ids
            -attn_masks : Tensor containing attention masks to be used to focus on non-padded values
            -token_type_ids : Tensor containing token type ids to be used to identify sentence1 and sentence2
        '''
        op = self.bert_layer(**{'input_ids':input_ids, 'attention_mask':attn_masks, 'token_type_ids':token_type_ids})
        logit = op.logits

        return logit

In [ ]:
def set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def calculate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct
    

def evaluate_loss(net, device, criterion, dataloader):
    net.eval()

    mean_loss = 0
    count = 0
    out_count = 0
    n_correct = 0
    with torch.no_grad():
        for it, batch in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = \
                batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['token_type_ids'].to(device), batch['labels'].to(device)
            logits = net(seq, attn_masks, token_type_ids)
            mean_loss += criterion(logits, labels).item()
            count += 1
            out_count += labels.size(0)
            big_val, big_idx = torch.max(logits.data, dim=1)
            n_correct += calculate_accuracy(big_idx, labels)
    
    avg_loss = mean_loss / count
    avg_acc = (n_correct*100)/out_count

    return avg_loss, avg_acc

In [ ]:
print("Creation of the models' folder...")
!mkdir models

Creation of the models' folder...


In [ ]:
def train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):
    loss_values = []
    best_loss = np.Inf
    best_ep = 1
    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5  # print the training loss 5 times per epoch
    iters = []
    train_losses = []
    val_losses = []

    #scaler = GradScaler()

    for ep in range(epochs):
        total_loss = 0.0

        net.train()
        running_loss = 0.0
        for it, batch in enumerate(tqdm(train_loader)):
            opti.zero_grad()
            # Converting to cuda tensors
            seq = batch['input_ids'].to(device)
            attn_masks = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)


            logits = net(seq, attn_masks, token_type_ids)
            
                # Computing loss
            loss = criterion(logits.squeeze(-1), labels)
            #loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged
            total_loss += loss.item()
            # Backpropagating the gradients
            # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
            #scaler.scale(loss).backward()
            loss.backward()                    #calculates gradients through backprop starting from loss
            torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)   #clip smaller gradients
            opti.step()             #updates parameters

        avg_train_loss = total_loss / len(train_loader)            
    
            # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
            
        print("")
        print("Running Validation...")
        
        net.eval()
        n_correct = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
        with torch.no_grad(): 
          for _, batch in enumerate(val_loader): 
            seq = batch['input_ids'].to(device)
            attn_masks = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            logits = net(seq, attn_masks, token_type_ids)
            loss = criterion(logits.squeeze(-1), labels)
            tr_loss += loss.item()

            big_val, big_idx = torch.max(logits.data, dim=1)
            n_correct += calculate_accuracy(big_idx, labels)

            nb_tr_steps += 1
            nb_tr_examples+=labels.size(0)


        epoch_loss = tr_loss/nb_tr_steps
        epoch_accu = (n_correct*100)/nb_tr_examples
        print("  Accuracy: {0:.2f}".format(epoch_accu))


## Parameters

In [ ]:
bert_model = "roberta-base"  # 'albert-base-v2', 'albert-large-v2', 'albert-xlarge-v2', 'albert-xxlarge-v2', 'bert-base-uncased', ...
freeze_bert = True #False  # if True, freeze the encoder weights and only update the classification layer weights
maxlen = 60  # maximum length of the tokenized input sentence pair : if greater than "maxlen", the input is truncated and else if smaller, the input is padded
bs = 8  # batch size
iters_to_accumulate = 2  # the gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate. If set to "1", you get the usual batch size
lr = 1e-5  # learning rate
epochs = 5  # number of training epochs

## Training and validation

In [ ]:
#  Set all seeds to make reproducible results
set_seed(1)

# Creating instances of training and validation set
print("Reading training data...")
train_set = CustomDataset(df_train, maxlen, bert_model)
# print("data issue: ", train_set.__getitem__(20))
print("Reading validation data...")
val_set = CustomDataset(df_val, maxlen, bert_model)
# Creating instances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size=bs, num_workers=5)
val_loader = DataLoader(val_set, batch_size=bs, num_workers=5)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = SentencePairClassifier(bert_model, freeze_bert=True)

if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)

net.to(device)

# criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()
opti = AdamW(net.parameters(), lr=lr, weight_decay=1e-2)
num_warmup_steps = 0 # The number of steps for the warmup phase.
num_training_steps = epochs * len(train_loader)  # The total number of training steps
t_total = (len(train_loader) // iters_to_accumulate) * epochs  # Necessary to take into account Gradient accumulation
lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate)

Reading training data...
Reading validation data...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly


  Average training loss: 1.31

Running Validation...


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

  Accuracy: 57.58


...

  Accuracy: 56.67


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si


  Average training loss: 0.31

Running Validation...



/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input s

  Accuracy: 57.17
